In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from __future__ import division

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import random
import cv2
import math
import numpy as np
import pandas as pd
import scipy
import scipy.misc
from scipy import pi
from subprocess import call
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt 
#import tensorflow as tf
from PIL import Image
import imageio

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
image_data = []
angle_data = []

# Get number of images
num_images = 0

# Number of images for training
num_train_images = 0

# Number of images for testing
num_test_images = 0

def load_dataset():
    # Read data.txt
    with open("C:/Users/npand/OneDrive/Desktop/MS_Course/EAS 509 Stats-Sakib/Project/data/driving_dataset/data.txt") as fp:
        for line in fp:
            image_data.append("C:/Users/npand/OneDrive/Desktop/MS_Course/EAS 509 Stats-Sakib/Project/data/driving_dataset/" + line.split()[0])

            # the paper by Nvidia uses the inverse of the turning radius,
            # but steering wheel angle is proportional to the inverse of turning radius
            # so the steering wheel angle in radians is used as the output       
            angle_data.append(float(line.split()[1]) * scipy.pi / 180)

def split_dataset(train_split,test_split):
    images_to_train = image_data[:int(len(image_data) * train_split)]    
    angles_to_train = angle_data[:int(len(image_data) * train_split)]

    images_to_test = image_data[-int(len(image_data) * test_split):]
    angles_to_test = angle_data[-int(len(image_data) * test_split):]
    
    return images_to_train,angles_to_train,images_to_test,angles_to_test


In [3]:
# Load dataset
load_dataset()

# Split dataset
images_to_train,angles_to_train,images_to_test,angles_to_test = split_dataset(0.8,0.2)

num_images = len(image_data)
print("Total number of images: ",num_images)

num_train_images = len(images_to_train)
print("Total number of images for training: ",num_train_images)

num_test_images = len(images_to_test)
print("Total number of images for testing: ",num_test_images)

Total number of images:  1501
Total number of images for training:  1200
Total number of images for testing:  300


In [4]:
#Model 1: Base line Model: y_test_pred = mean(y_train_i) 
train_mean_angle = np.mean(angles_to_train)

print('Test_MSE(MEAN):%f' % np.mean(np.square(angles_to_test - train_mean_angle)))

Test_MSE(MEAN):1.809630


In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

In [6]:
# true_image = tf.placeholder(tf.float32, shape=[None, 66, 200, 3],name="true_image")
# true_angle = tf.placeholder(tf.float32, shape=[None, 1],name="true_angle")

# x_image = true_image
# print("True Imag",true_image.shape)
# #first convolutional layer
# W_conv1 = weight_variable([5, 5, 3, 24])
# b_conv1 = bias_variable([24])

# h_conv1= tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)
# print("Sa",h_conv1.shape)

# #second convolutional layer
# W_conv2 = weight_variable([5, 5, 24, 36])
# b_conv2 = bias_variable([36])

# h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)
# print("Sa",h_conv2.shape)
# #third convolutional layer
# W_conv3 = weight_variable([3, 3, 36, 48])
# b_conv3 = bias_variable([48])

# h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)
# print("Sa3",h_conv3.shape)

# h_conv3_max=tf.compat.v1.nn.max_pool(h_conv3,ksize=[1,2,2,1],strides=[1, 2, 2, 1],padding='VALID')
# print("Sa_Max3",h_conv3_max.shape)

# #fourth convolutional layer
# W_conv4 = weight_variable([3, 3, 48, 64])
# b_conv4 = bias_variable([64])

# h_conv4 = tf.nn.relu(conv2d(h_conv3_max, W_conv4, 1) + b_conv4)
# print("Sa4",h_conv4.shape)

# # #fifth convolutional layer
# # W_conv5 = weight_variable([3, 3, 64, 64])
# # b_conv5 = bias_variable([64])

# # h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)
# # print("Sa5",h_conv5.shape)

# #FCL 1
# W_fc1 = weight_variable([576, 502])
# b_fc1 = bias_variable([502])

# h_conv4_flat = tf.reshape(h_conv4, [-1, 576])
# print("Flat1",h_conv4_flat.shape)

# h_fc1 = tf.nn.relu(tf.matmul(h_conv4_flat, W_fc1) + b_fc1)

# keep_prob = tf.placeholder(tf.float32,name="keep_prob")
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# print("Flat2",h_fc1.shape)
# #FCL 2
# W_fc2 = weight_variable([502, 100])
# b_fc2 = bias_variable([100])

# h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

# #FCL 3
# W_fc3 = weight_variable([100, 50])
# b_fc3 = bias_variable([50])#FCL 3
# W_fc3 = weight_variable([100, 50])
# b_fc3 = bias_variable([50])

# h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

# h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

# #FCL 4
# W_fc4 = weight_variable([50, 10])
# b_fc4 = bias_variable([10])

# h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

# h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

# #Output
# W_fc5 = weight_variable([10, 1])
# b_fc5 = bias_variable([1])

# # activation function with scaling
# predicted_angle = tf.multiply(tf.proj(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2)
# predicted_angle = tf.identity(predicted_angle,name="predicted_angle")

In [7]:
LOGDIR = './models/proj/'

# Lets start the tensorflow session
sess = tf.InteractiveSession()

In [8]:
true_image = tf.placeholder(tf.float32, shape=[None, 66, 200, 3],name="true_image")
true_angle = tf.placeholder(tf.float32, shape=[None, 1],name="true_angle")

x_image = true_image

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32,name="keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 4
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

# activation function with scaling
predicted_angle = tf.multiply(tf.atan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2)
predicted_angle = tf.identity(predicted_angle,name="predicted_angle")


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
#points to the end of the last batch
train_batch_pointer = 0
test_batch_pointer = 0

# Utility Functions
def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        #x_out.append(scipy.misc.imresize(scipy.misc.imread(images_to_train[(train_batch_pointer + i) % num_train_images])[-150:], 
        #                                 [66, 200]) / 255.0)
        x_out.append(np.array(Image.fromarray((imageio.imread(images_to_train[(train_batch_pointer + i) % num_train_images])[-150:])).resize((200, 66),resample=Image.BILINEAR))/ 255.0)
        a=np.array(Image.fromarray((imageio.imread(images_to_train[(train_batch_pointer + i) % num_train_images])[-150:])).resize((200, 66),resample=Image.BILINEAR))
        #print("Hi",i, a.shape)
        y_out.append([angles_to_train[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadTestBatch(batch_size): 
    global test_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(np.array(Image.fromarray((imageio.imread(images_to_test[(test_batch_pointer + i) % num_test_images])[-150:])).resize((200, 66),resample=Image.BILINEAR))/ 255.0)

        y_out.append([angles_to_test[(test_batch_pointer + i) % num_test_images]])
    test_batch_pointer += batch_size
    return x_out, y_out

In [10]:
start = datetime.now()

print("Let the model learn itself...")
print()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(true_angle, predicted_angle))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)

# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver()

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 16
batch_size = 50

# train over the dataset about 30 times
previous_i = 0
previous_loss = 0
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):        
        xs, ys = LoadTrainBatch(batch_size)
        #print(xs)
        train_step.run(feed_dict={true_image: xs, true_angle: ys, keep_prob: 0.80})
        if i % 10 == 0:            
            xs, ys = LoadTestBatch(batch_size)
            loss_value = loss.eval(feed_dict={true_image:xs, true_angle: ys, keep_prob: 1.0})
            previous_loss = loss_value
            previous_i = i
            # print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={true_image:xs, true_angle: ys, keep_prob: 1.0})
        summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)            
            checkpoint_path = os.path.join(LOGDIR, "model_proj.ckpt")
            filename = saver.save(sess, checkpoint_path)    
    print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + previous_i, previous_loss)) 
    print("Model saved in file: %s" % filename)
    print()

# print("Run the command line:\n" \
#           "--> tensorboard --logdir=./logs " \
#           "\nThen open http://0.0.0.0:6006/ into your web browser")

print("\nTime taken to train the model: ",datetime.now() - start)


Let the model learn itself...

Epoch: 0, Step: 20, Loss: 7.91695
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 1, Step: 70, Loss: 6.07974
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 2, Step: 120, Loss: 7.58821
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 3, Step: 170, Loss: 5.79875
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 4, Step: 220, Loss: 7.40211
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 5, Step: 270, Loss: 5.59238
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 6, Step: 320, Loss: 7.08665
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 7, Step: 370, Loss: 5.37652
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 8, Step: 420, Loss: 7.46914
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 9, Step: 470, Loss: 5.2594
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 10, Step: 520, Loss: 6.52543
Model saved in file: ./models/proj/model_proj.ckpt

Epoch: 11, Ste

## VGG NET 16

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [12]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(66,200,3))
base_model.trainable = False ## Not trainable weights

base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 66, 200, 3)]      0         
                                                                 
 block1_conv1 (Conv2D)       (None, 66, 200, 64)       1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 66, 200, 64)       36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 33, 100, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 33, 100, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 33, 100, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 50, 128)       0     

In [13]:

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1152, activation='relu')
dense_layer_2 = layers.Dense(100, activation='relu')
dense_layer_3 = layers.Dense(50, activation='relu')
dense_layer_4 = layers.Dense(10, activation='relu')
prediction_layer = layers.Dense(1, activation='linear')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    dense_layer_4,
    prediction_layer
])


In [14]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

xs,ys=LoadTrainBatch(1500)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(np.array(xs), np.array(ys).reshape(-1,1), epochs=5, validation_split=0.2, batch_size=32, callbacks=[es])

Train on 1200 samples, validate on 300 samples
Epoch 1/5
1200/1200 [==============================] - ETA: 0s - loss: 1.6154

C:\Users\npand\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


1200/1200 [==============================] - 103s 86ms/sample - loss: 1.6154 - val_loss: 0.8391
Epoch 2/5
1200/1200 [==============================] - 88s 74ms/sample - loss: 0.3076 - val_loss: 0.4313
Epoch 3/5
1200/1200 [==============================] - 105s 87ms/sample - loss: 0.1677 - val_loss: 0.2837
Epoch 4/5
1200/1200 [==============================] - 112s 93ms/sample - loss: 0.1565 - val_loss: 0.2849
Epoch 5/5
1200/1200 [==============================] - 126s 105ms/sample - loss: 0.1496 - val_loss: 0.2933


## RESTNET50

In [15]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [16]:
## Loading RestNet50 model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(66,200,3))
base_model.trainable = False ## Not trainable weights

base_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 66, 200, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 72, 206, 3)   0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 33, 100, 64)  9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNor

Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
__________________________________________________________________________________________________


In [17]:

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1152, activation='relu')
dense_layer_2 = layers.Dense(100, activation='relu')
dense_layer_3 = layers.Dense(50, activation='relu')
dense_layer_4 = layers.Dense(10, activation='relu')
prediction_layer = layers.Dense(1, activation='linear')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    dense_layer_4,
    prediction_layer
])


In [18]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

xs,ys=LoadTrainBatch(1500)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(np.array(xs), np.array(ys).reshape(-1,1), epochs=5, validation_split=0.2, batch_size=32, callbacks=[es])

Train on 1200 samples, validate on 300 samples
Epoch 1/5
1200/1200 [==============================] - 64s 53ms/sample - loss: 2.7743 - val_loss: 2.9312
Epoch 2/5
1200/1200 [==============================] - 57s 48ms/sample - loss: 0.3550 - val_loss: 3.0272
Epoch 3/5
1200/1200 [==============================] - 53s 44ms/sample - loss: 0.1333 - val_loss: 3.0807
Epoch 4/5
1200/1200 [==============================] - 50s 42ms/sample - loss: 0.1052 - val_loss: 3.3440
Epoch 5/5
1200/1200 [==============================] - 53s 44ms/sample - loss: 0.0900 - val_loss: 3.0491


## InceptionV3

In [19]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [20]:
#points to the end of the last batch
train_batch_pointer = 0
test_batch_pointer = 0

# Utility Functions
def LoadTrainBatch_Inception(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        #x_out.append(scipy.misc.imresize(scipy.misc.imread(images_to_train[(train_batch_pointer + i) % num_train_images])[-150:], 
        #                                 [66, 200]) / 255.0)
        x_out.append(np.array(Image.fromarray((imageio.imread(images_to_train[(train_batch_pointer + i) % num_train_images])[-150:])).resize((200, 75),resample=Image.BILINEAR))/ 255.0)
        a=np.array(Image.fromarray((imageio.imread(images_to_train[(train_batch_pointer + i) % num_train_images])[-150:])).resize((200, 75),resample=Image.BILINEAR))
        #print("Hi",i, a.shape)
        y_out.append([angles_to_train[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

In [21]:
## Loading InceptionV3 model
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(75,200,3))
base_model.trainable = False ## Not trainable weights

base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 75, 200, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 37, 99, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 37, 99, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [22]:

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(1152, activation='relu')
dense_layer_2 = layers.Dense(100, activation='relu')
dense_layer_3 = layers.Dense(50, activation='relu')
dense_layer_4 = layers.Dense(10, activation='relu')
prediction_layer = layers.Dense(1, activation='linear')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    dense_layer_4,
    prediction_layer
])


In [23]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

xs,ys=LoadTrainBatch_Inception(1500)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(np.array(xs), np.array(ys).reshape(-1,1), epochs=5, validation_split=0.2, batch_size=32, callbacks=[es])

Train on 1200 samples, validate on 300 samples
Epoch 1/5
1200/1200 [==============================] - 16s 13ms/sample - loss: 2.4362 - val_loss: 4.1662
Epoch 2/5
1200/1200 [==============================] - 19s 16ms/sample - loss: 1.0235 - val_loss: 3.1026
Epoch 3/5
1200/1200 [==============================] - 20s 17ms/sample - loss: 0.6345 - val_loss: 3.2609
Epoch 4/5
1200/1200 [==============================] - 45s 37ms/sample - loss: 0.7023 - val_loss: 3.5585
Epoch 5/5
1200/1200 [==============================] - 39s 33ms/sample - loss: 0.5843 - val_loss: 3.8672
